# Prepare River Radar Dataset

1. Download `rain_radar.tar.gz (58MB)` from https://datasets.cms.waikato.ac.nz/taiao/river_radar_2015_2018/.
2. Once obtained please place the archive in your `TORCH_DATA_DIR` directory. **DO NOT EXTRACT THE ARCHIVE.**
3. Run this notebook to generate `riverradar.hdf5`.

In [ ]:
import torch
import pandas as pd
from tarfile import TarFile
import numpy as np
from pathlib import Path
from os import environ
from sklego.preprocessing import RepeatingBasisFunction
import h5py

data_directory = Path(environ.get("TORCH_DATA_DIR", ".")).expanduser().resolve()
riverradar_data = data_directory / "river_radar.tar.gz"
with TarFile.open(riverradar_data) as tar:
    df = pd.read_csv(tar.extractfile("merged_data.csv"))

## Build Time Features
Neural networks require timestamps to be embedded into real numbers. We do
this using repeating basis functions as described in (1).

- (1) https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/

In [ ]:
datetimes = pd.to_datetime(df["datetimeUTC"])
year_offset = datetimes.apply(lambda datetime: datetime.year - 2015)
day_of_year = datetimes.apply(lambda datetime: datetime.dayofyear)
hour_of_day = datetimes.apply(lambda datetime: datetime.hour)

In [ ]:
def rbf_day_of_year(day_of_year: np.ndarray) -> np.ndarray:
    df_day_of_year = pd.DataFrame({"day_of_year": day_of_year})
    rbf_day_of_year = RepeatingBasisFunction(
        n_periods=12, input_range=(1, 365), remainder="drop", column="day_of_year"
    )
    rbf_day_of_year.fit(df_day_of_year)
    return rbf_day_of_year.transform(df_day_of_year)


def rbf_hour_of_day(hour_of_day: np.ndarray) -> np.ndarray:
    df_hour_of_day = pd.DataFrame({"hour_of_day": hour_of_day})
    rbf_hour_of_day = RepeatingBasisFunction(
        n_periods=6, input_range=(0, 23), remainder="drop", column="hour_of_day"
    )
    rbf_hour_of_day.fit(df_hour_of_day)
    return rbf_hour_of_day.transform(df_hour_of_day)


with torch.no_grad():
    day_embedding = torch.tensor(rbf_day_of_year(day_of_year))
    hour_embedding = torch.tensor(rbf_hour_of_day(hour_of_day))
    year_embedding = torch.nn.functional.one_hot(
        torch.tensor(year_offset), num_classes=4
    )
    time_features = torch.cat([day_embedding, hour_embedding, year_embedding], dim=1)

In [ ]:
for i, feature in enumerate(df.columns):
    print(i, feature)

### Other Features
 * Columns 1-140 contains the gridded radar reflectance taken at 10x14 grid, taken at sea level
 * Columns 141-280 contains the gridded radar reflectance taken at 10x14 grid, taken at 2000m above sea level
 * Columns 281-420 contains the gridded radar reflectance taken at 10x14 grid, taken at 4000m above sea level
 * Columns 421-422 contains the river stage levels

In [ ]:
features = torch.tensor(df[df.columns[1:421]].to_numpy())
targets = torch.tensor(df[df.columns[421:]].to_numpy())
print(features.shape, targets.shape)

### Combine Features and Standardize

In [ ]:
x_dataset = torch.cat([features, time_features], dim=1)
y_dataset = targets

# Standardize the features
x_dataset = (x_dataset - x_dataset.mean(dim=0)) / x_dataset.std(dim=0)
y_dataset = (y_dataset - y_dataset.mean(dim=0)) / y_dataset.std(dim=0)

print("x_dataset.shape:", x_dataset.shape)
print("y_dataset.shape:", y_dataset.shape)
# Count NaNs in the dataset
print("Number of NaNs in x_dataset:", np.isnan(x_dataset).sum())
print("Number of NaNs in y_dataset:", np.isnan(y_dataset).sum())

x_dataset = x_dataset.numpy().astype(np.float32)
x_dataset = np.nan_to_num(x_dataset, nan=0.0)
y_dataset = y_dataset.numpy().astype(np.float32)

In [ ]:
hdf5_filename = data_directory / "riverradar.hdf5"

with h5py.File(hdf5_filename, "w") as f:
    f.create_dataset("x_features", data=x_dataset, dtype="float32", compression="gzip")
    f.create_dataset("y_targets", data=y_dataset, dtype="float32", compression="gzip")